# MoMa's collection data cleaning

## Problem solving
I'm working for the MoMa and they'd like to know in which department they need to enrich based on the current collection. 

**What is the Top-3 less valuable classification?**

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df=pd.read_csv('data/museum_modern_art.csv',sep=',')
df.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,...,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,...,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,...,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,...,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,...,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


In [3]:
df.dtypes

Unnamed: 0             object
Title                  object
Artist                 object
ConstituentID          object
ArtistBio              object
Nationality            object
BeginDate              object
EndDate                object
Gender                 object
Date                   object
Medium                 object
Dimensions             object
CreditLine             object
AccessionNumber        object
Classification         object
Department             object
DateAcquired           object
Cataloged              object
ObjectID               object
URL                    object
ThumbnailURL           object
Circumference (cm)    float64
Depth (cm)            float64
Diameter (cm)         float64
Height (cm)           float64
Length (cm)           float64
Weight (kg)           float64
Width (cm)            float64
Seat Height (cm)      float64
Duration (sec.)       float64
dtype: object

## Renaming columns

In [4]:
df1=df.rename(columns={'Unnamed: 0':'Id'})

In [5]:
df1.columns

Index(['Id', 'Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

## Drop empty and useless tables

In [6]:
null_col=df1.isna().sum()
null_col_percent=round(null_col[null_col>0]/df1.shape[0]*100,2)
null_col_percent

Title                   0.03
Artist                 12.20
ConstituentID          12.20
ArtistBio              14.81
Nationality            12.20
BeginDate              12.20
EndDate                12.20
Gender                 12.20
Date                   12.84
Medium                 18.64
Dimensions             18.40
CreditLine             13.14
AccessionNumber        11.26
Classification         11.26
Department             11.26
DateAcquired           15.62
Cataloged              11.26
ObjectID               11.26
URL                    49.17
ThumbnailURL           56.12
Circumference (cm)     99.99
Depth (cm)             91.45
Diameter (cm)          99.07
Height (cm)            23.75
Length (cm)            99.52
Weight (kg)            99.81
Width (cm)             24.34
Seat Height (cm)      100.00
Duration (sec.)        97.93
dtype: float64

In [7]:
drop_cols=null_col_percent[null_col_percent>50].index
df2=df1.drop(drop_cols,axis=1)

In [8]:
df2.shape

(152487, 22)

## Remove Duplicates

In [9]:
df3=df2.copy()
df3.iloc[:,:].duplicated().sum()

17168

In [10]:
print(df3.iloc[:,:].shape)
df4=df3.iloc[:,:].drop_duplicates()
print(df4.iloc[:,:].shape)

(152487, 22)
(135319, 22)


# Put off parenthesis on text in relevant columns

In [11]:
parenthesis_col=['ArtistBio','Nationality','BeginDate','EndDate','Gender','Date']
parenthesis_col

df5=df4.copy()

In [12]:
for col in parenthesis_col:
    df5[col]=df5[col].str.replace('\(','').str.replace('\)','')
    
df5.head()

,Id,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,...,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,Height (cm),Width (cm)
0,0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"Austrian, 1841–1918",Austrian,1841,1918,Male,1896,...,Fractional and promised gift of Jo Carole and ...,885.1996,Architecture,Architecture & Design,1996-04-09,Y,2,http://www.moma.org/collection/works/2,48.6000,168.9000
1,1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"French, born 1944",French,1944,0,Male,1987,...,Gift of the architect in honor of Lily Auchinc...,1.1995,Architecture,Architecture & Design,1995-01-17,Y,3,http://www.moma.org/collection/works/3,40.6401,29.8451
2,2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"Austrian, 1876–1957",Austrian,1876,1957,Male,1903,...,Gift of Jo Carole and Ronald S. Lauder,1.1997,Architecture,Architecture & Design,1997-01-15,Y,4,http://www.moma.org/collection/works/4,34.3000,31.8000
3,3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"French and Swiss, born Switzerland 1944",,1944,0,Male,1980,...,Purchase and partial gift of the architect in ...,2.1995,Architecture,Architecture & Design,1995-01-17,Y,5,http://www.moma.org/collection/works/5,50.8000,50.8000
4,4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"Austrian, 1876–1957",Austrian,1876,1957,Male,1903,...,Gift of Jo Carole and Ronald S. Lauder,2.1997,Architecture,Architecture & Design,1997-01-15,Y,6,http://www.moma.org/collection/works/6,38.4000,19.1000


# Clean Date values

In [72]:
df6=df5.copy()
print(df6.Date.unique())
start_values=df6.Date.nunique()
print("total unique values in date: ",start_values)

['1896' '1987' '1903' ... '1961-1962' 'early 1980s' '1979–1983']
total unique values in date:  8815


In [73]:
df6.Date=df6.Date.astype(str)

In [74]:
def test(date):
    count_not_str=0
    if type(date) != str:
        count_not_str+=1
    return count_not_str

# Check if convert is working
count_type=df6.Date.apply(test).value_counts()
count_type

0    135319
Name: Date, dtype: int64

In [106]:
def clean_date(date):
    if re.search('.[0-9]{4}$', date):
        return date[-4:]
    if re.search('^[0-9]{4}.', date):
        return date[:4]
    elif re.search('.[0-9]{4}.', date):
        pos = re.search('[0-9]{4}', date).start()
        return date[pos:pos+4]
    elif re.search('[Uu]nk[n]*own', date):
        return np.nan
    elif re.search('[0-9]{3}\?', date):
        new_date = re.sub('\?','0',date)
        pos = re.search('[0-9]{4}', new_date).start()
        return new_date[pos:pos+4]
    elif re.search('^[a-zA-Z \,\?]+$', date):
        return "only letters"
    else:
        return date
    
    
# Testing function
date='Lithograph with screenprinted binder'
new_date=clean_date(date)
print(new_date)

only letters


In [82]:
df8=df6.copy()
before_clean2=df8.Date.nunique()
print("total unique values in date before clean 2: ", before_clean2)

df8.Date=df8.Date.apply(clean_date)

print(df8.Date.value_counts())
print(df8.Date.unique())
clean2_values=df8.Date.nunique()
print("total unique values in date after clean 2: ", clean2_values)

total unique values in date before clean 2:  8816
1966                    2538
1967                    2472
nan                     2406
1969                    2405
1965                    2386
                        ... 
1830                       1
c. 196?                    1
1635                       1
1848                       1
7th-8th century C.E.       1
Name: Date, Length: 218, dtype: int64
['1896' '1987' '1903' '1980' '1976' '1968' '1900' '1978' '1905' '1906'
 '1979' '1918' '1970' '1975' '1984' '1986' '1974' 'n.d.' '1917' '1923' nan
 '1930' '1936' '1935' '1937' '1938' '1977' '1958' '1985' '1989' '1949'
 '1964' '1991' '1941' '1965' '1981' '1983' '1988' '1992' '1915' '1953'
 '1910' '1982' '1945' '1924' '1990' '1995' '1931' '1929' '1959' '1920'
 '1939' '1993' '1996' '1952' '1921' '1957' '1972' '1956' '1962' '1925'
 '1960' '1969' '1963' '1994' '1961' '1928' '1927' '1933' '1967' '1934'
 '1940' '1946' '1955' '1997' '1922' '1942' '1954' '1973' '1926' '1932'
 '1947' '1943' '1944'

In [46]:
df6.loc[:,'Date']=df6.loc[:,'Date'].str.replace("'",'').str.replace('.','').str.replace('early','').str.replace('s','').str.replace('c.','').str.replace('After','').str.replace('or before','').str.replace(' publihed','').str.replace('printed ','').str.replace('newpaperSeptember','').str.replace('exeted','').str.replace('Before','').str.replace(' ','')
print(df6.Date.unique())
clean1_values=df6.Date.nunique()
print("total unique values in date after clean 1: ", clean1_values)


['1896' '1987' '1903' ... '1896-1903' '1961-1962' '1979–1983']
total unique values in date after clean 1:  8236
